<a href="https://colab.research.google.com/github/Ihsan101/FreeCodeCamp-final-projects/blob/main/Book%20Recommendation%20Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-09-11 07:04:07--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   157MB/s    in 0.2s    

2023-09-11 07:04:07 (157 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
#Resources used. I had referred to multiple github pages on solutions given by previous students to understand the code better. I havent copy pasted any code directory, either rewritten after understanding what they did or tried my own method to do the same.
#Any staff reading this, if possible give a page for helpful resources or pages which we can visit to learn more about to learn more about the machine learning model we are working on.
#Also update the course material. The estimators used in the start of the models are now deprecated and arent maintained. They might be removed in future updates, and are unpredictable tbh.

#Derick Mitey has provided a lot of assistance for me to complete this model. Huge thanks to him

#Grouping by user gives and then using count of rating gives total count of that particular author.
#Grouping by isbn and using the same command can give us the count of the reviews of the book.
user_RatingCount = df_ratings.groupby('user')['rating'].count().reset_index().rename(columns = {'rating':'UserRatings'})
book_RatingCount = df_ratings.groupby('isbn')['rating'].count().reset_index().rename(columns = {'rating':'BookRatings'})
#I learnt this method to go through a panda database after finding it in a github page of a user, then searching about it on StackOverflow.

# Add to df_ratings
df_ratings = df_ratings.merge(user_RatingCount,how='left')
df_ratings = df_ratings.merge(book_RatingCount, how='left')

#Filter data so users with less than 200 ratings and books with less than 100 ratings isnt included.

df_ratings =df_ratings.loc[(df_ratings['UserRatings']>=200)]
df_ratings =df_ratings.loc[(df_ratings['BookRatings']>=100)]
#Merging both databases, with ratings columns on the right side and the books details on the left side, with the common column
books_ratings = pd.merge(right = df_ratings, left = df_books, on='isbn')

#Experienced issues with duplicate data during testing the code. Not sure how it occurs, but this line removes such repeated data.
books_ratings = books_ratings.drop_duplicates(['title', 'user'])

df = books_ratings.pivot_table(values='rating', index='title', columns='user').fillna(0)

df_matrix = csr_matrix(df.values)

model = NearestNeighbors(algorithm='auto', metric='cosine', n_neighbors = 6)
model.fit(df_matrix)


NearestNeighbors(metric='cosine', n_neighbors=6)

In [ ]:
# function to return recommended books - this will be tested

#Added a manual mode in case we want to input the name of the book during runtime and check it against the model to get new reccomendations
def get_recommends(book = "", manual = False):
  if manual:
    book = input("Enter name of book to search for in the database:\n")

  #Runs if the book defined by the user doesnt exist.
  try:
    df.loc[book]
  except KeyError:
    print(f"Given book,{book}, does not exist")

  #Get the distance and index of the reccomended books using the model
  distance, index = model.kneighbors([df.loc[book]], return_distance = True)

  #Final reccomendation list, which we shall append later on with the values to be displayed
  final_recc = []

  #Reversing order of the index, since we need the most likely reccomendation first, and the following books should be less likely than the first
  for i in range(5,0,-1):

      #Flattening the index and distance array into 1D so we can properly index through the list. Taking the i index value which is defined by the iteration of the loop, and then turn the integer value of the name of the book back to string using df.index
      final_recc.append([df.index[index.flatten()[i]], distance.flatten()[i]])
  #Append values from the loop into final list to be displayed
  final_recc = [book, final_recc]

  return final_recc

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
